In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/nlp-getting-started/train.csv'

In [ ]:
df1 = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
df1.head(3)

In [4]:
#making all the letters to lowercase
df["text"] = df["text"].apply(lambda x: x.lower())
df1["text"] = df1["text"].apply(lambda x: x.lower())

NameError: name 'df' is not defined

In [5]:
import re

In [7]:
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

df["text"] = df["text"].apply(lambda x: remove_URL(x))
df1["text"] = df1["text"].apply(lambda x: remove_URL(x))

In [8]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

df["text"] = df["text"].apply(lambda x: remove_html(x))
df1["text"] = df1["text"].apply(lambda x: remove_html(x))

In [9]:
import string
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))


df["text"] = df["text"].apply(lambda x: remove_punct(x))
df1["text"] = df1["text"].apply(lambda x: remove_punct(x))

In [10]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [11]:
df1.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [12]:
df = df.drop('keyword', axis=1)
df1 = df1.drop('keyword', axis=1)

In [13]:
df = df.drop('location', axis=1)
df1 = df1.drop('location', axis=1)

In [14]:
import nltk
from nltk import TweetTokenizer

tokenizer = TweetTokenizer()

df['tokens'] = [tokenizer.tokenize(item) for item in df.text]
df1['tokens'] = [tokenizer.tokenize(item) for item in df1.text]

In [15]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_item(item):
    new_item = []
    for x in item:
        x = lemmatizer.lemmatize(x)
        new_item.append(x)
    return " ".join(new_item)

df['tokens'] = [lemmatize_item(item) for item in df.tokens]
df1['tokens'] = [lemmatize_item(item) for item in df1.tokens]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


X = vectorizer.fit_transform(df.text).toarray()
y = df['target']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [18]:
from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(X_train,y_train)
y_pred_MNB = mb.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [20]:
acc_MNB = accuracy_score(y_test,y_pred_MNB)
print("The accuracy of the model is :",acc_MNB)

The accuracy of the model is : 0.8025394045534151


In [21]:
cf_MNB = confusion_matrix(y_test,y_pred_MNB)
cf_MNB


array([[1157,  181],
       [ 270,  676]])

In [22]:
print(classification_report(y_test,y_pred_MNB))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1338
           1       0.79      0.71      0.75       946

    accuracy                           0.80      2284
   macro avg       0.80      0.79      0.79      2284
weighted avg       0.80      0.80      0.80      2284



In [23]:
from sklearn.preprocessing import StandardScaler 
ss=StandardScaler() 
X_train_ss=ss.fit_transform(X_train)
X_test_ss=ss.transform(X_test)

In [24]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr_model=lr.fit(X_train_ss,y_train)
lr_model

LogisticRegression()

In [25]:
lr_y_pred=lr_model.predict(X_test_ss)

In [26]:
from sklearn.metrics import accuracy_score
log_acc=accuracy_score(y_test,lr_y_pred)
log_acc

0.7788966725043783

In [27]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr_model=lr.fit(X_train,y_train)
lr_model

LogisticRegression()

In [28]:
lr_y_pred=lr_model.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
log_acc=accuracy_score(y_test,lr_y_pred)
log_acc

0.8025394045534151

In [30]:
from sklearn.svm import LinearSVC
LinSVC=LinearSVC()
model_SVC=LinSVC.fit(X_train_ss,y_train)
LinSVC_y_pred=model_SVC.predict(X_test_ss)
LinSVC_acc=accuracy_score(y_test,LinSVC_y_pred)
LinSVC_acc

0.760507880910683

In [31]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn_model=knn.fit(X_train_ss,y_train)
knn_y_pred=knn_model.predict(X_test_ss)
knn_acc=accuracy_score(y_test,knn_y_pred)
knn_acc

0.6571803852889667

In [32]:
#multinomialNB gives better accuracy

In [33]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [34]:
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [35]:
test = vectorizer.transform(df1.text).toarray()

In [36]:
pred_1 = mb.predict(test)

In [37]:
sub['target'] = pred_1
sub.to_csv('submission.csv',index=False)

In [38]:
r =pd.read_csv("submission.csv")
r

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
